# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,newnham,-41.3833,147.1333,292.68,45,40,9.26,AU,1733880914
1,1,port-aux-francais,-49.3500,70.2167,277.44,69,75,7.34,TF,1733880915
2,2,adamstown,-25.0660,-130.1015,296.24,81,85,7.52,PN,1733880917
3,3,khromtau,50.2503,58.4347,265.29,89,100,5.93,KZ,1733880918
4,4,tiksi,71.6872,128.8694,241.73,100,52,1.14,RU,1733880919


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity',
    color= 'City'    
)
# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
vacay1 = city_data_df[city_data_df['Humidity'] > 70]
vacay2 = vacay1[vacay1['Cloudiness'] < 50]
vacay3 = vacay2[vacay2['Wind Speed'] < 5 ]
# Drop any rows with null values
# YOUR CODE HERE
vacay_clean = vacay3.dropna()
# Display sample data
# YOUR CODE HERE
vacay_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,bredasdorp,-34.5322,20.0403,291.78,93,2,2.04,ZA,1733880922


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = vacay_clean[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,bredasdorp,ZA,-34.5322,20.0403,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    "categories": "beach.beach_resort",
    "limit":1,
    "apiKey":geoapify_key    }
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame

for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    lon = row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    params["lat"] = lat
    params["lon"] = lon
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address2 = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address2["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
malabar - nearest hotel: Alexandra Headland Beach Resort (Break Free)
hawaiian paradise park - nearest hotel: Sands of Kahana Resort
port elizabeth - nearest hotel: The Willows Resort
bethel - nearest hotel: No hotel found
ulanhot - nearest hotel: 九站沙滩
chinde - nearest hotel: Tripura Resort Pvt.Ltd
vilyuchinsk - nearest hotel: No hotel found
talnakh - nearest hotel: Озеро светлое
shingu - nearest hotel: 加太海水浴場
alofi - nearest hotel: Saletoga Sands Resort & Spa
iqaluit - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Grand Anse
ilulissat - nearest hotel: Badestelle
blackmans bay - nearest hotel: Alexandra Headland Beach Resort (Break Free)
uturoa - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
igrim - nearest hotel: No hotel found
bredasdorp - nearest hotel: Stilbaai Lofts
waitangi - nearest hotel: Castaway Resort
bilibino - nearest hotel: No hotel found
constantia -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,malabar,AU,-33.9667,151.2500,80,Alexandra Headland Beach Resort (Break Free)
2,hawaiian paradise park,US,19.5933,-154.9731,83,Sands of Kahana Resort
3,port elizabeth,ZA,-33.9180,25.5701,96,The Willows Resort
4,bethel,US,41.3712,-73.4140,85,No hotel found
5,ulanhot,CN,46.0833,122.0833,74,九站沙滩
...,...,...,...,...,...,...
562,kertih,MY,4.5141,103.4483,94,Strawberry Kijal Resort Apartments
563,mooirivier,ZA,-29.2082,29.9946,94,La Côte d’Azur
564,woolgoolga,AU,-30.1106,153.2007,91,Alexandra Headland Beach Resort (Break Free)
567,corner brook,CA,48.9667,-57.9484,92,Pasadena Beach Provincial Park


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 'Humidity'    


# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)